In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, AveragePooling2D, Flatten, Dense


In [2]:
(xtrain,ytrain),(xtest,ytest)= keras.datasets.mnist.load_data()
xtrain=np.dstack([xtrain] * 3)
xtest=np.dstack([xtest]*3)

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
xtrain = xtrain.reshape(-1, 28,28,3)
xtest= xtest.reshape (-1,28,28,3)
xtrain.shape

(60000, 28, 28, 3)

In [4]:
from keras.preprocessing.image import img_to_array, array_to_img

xtrain = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in xtrain])
xtest = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in xtest])

In [5]:
from tensorflow.keras.applications import ResNet50
input_layer=layers.Input(shape=(48,48,3))
model_ResNet50=ResNet50(weights='imagenet',input_tensor=input_layer,include_top=False)

model_ResNet50.summary()

94765736/94765736 [==============================] - 5s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 54, 54, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 24, 24, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 24, 24, 64)           256       ['conv1_conv[0][0]']          
 on)                      

In [6]:
for layer in model_ResNet50.layers:
    layer.trainable = False
model = models.Sequential()
model.add(model_ResNet50)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2 = model.fit(xtrain, ytrain,batch_size=128, epochs=10,
                    validation_data=(xtest, ytest))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


469/469 [==============================] - 30s 40ms/step - loss: 0.4607 - accuracy: 0.8579 - val_loss: 0.2540 - val_accuracy: 0.9201
Epoch 2/10
469/469 [==============================] - 16s 34ms/step - loss: 0.2335 - accuracy: 0.9277 - val_loss: 0.2144 - val_accuracy: 0.9306
Epoch 3/10
469/469 [==============================] - 16s 35ms/step - loss: 0.1923 - accuracy: 0.9402 - val_loss: 0.1895 - val_accuracy: 0.9377
Epoch 4/10
469/469 [==============================] - 16s 34ms/step - loss: 0.1681 - accuracy: 0.9479 - val_loss: 0.1625 - val_accuracy: 0.9485
Epoch 5/10
469/469 [==============================] - 16s 35ms/step - loss: 0.1558 - accuracy: 0.9501 - val_loss: 0.1663 - val_accuracy: 0.9457
Epoch 6/10
469/469 [==============================] - 16s 35ms/step - loss: 0.1442 - accuracy: 0.9533 - val_loss: 0.1433 - val_accuracy: 0.9532
Epoch 7/10
469/469 [==============================] - 16s 35ms/step - loss: 0.1338 - accuracy: 0.9577 - val_loss: 0.1459 - val_accuracy: 0.9524
Epo

In [8]:
def identity_block(X, filters):
    # Implementation of the identity block

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of the main path
    X = Conv2D(F1, (1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Second component of the main path
    X = Conv2D(F2, (3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Third component of the main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X


In [9]:
def convolutional_block(X, filters, s=2):
    # Implementation of the convolutional block

    F1, F2, F3 = filters

    X_shortcut = X

    # First component of the main path
    X = Conv2D(F1, (1, 1), strides=(s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Second component of the main path
    X = Conv2D(F2, (3, 3), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)

    # Third component of the main path
    X = Conv2D(F3, (1, 1), strides=(1, 1))(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut to the main path
    X = Add()([X, X_shortcut])
    X = ReLU()(X)

    return X


In [10]:
def ResNet50(input_shape=(48, 48, 3), classes=10):
    # Implementation of the ResNet-50 architecture

    X_input = Input(input_shape)

    # Zero-padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2))(X)
    X = BatchNormalization(axis=3)(X)
    X = ReLU()(X)
    X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, filters=[64, 64, 256], s=1)
    X = identity_block(X, filters=[64, 64, 256])
    X = identity_block(X, filters=[64, 64, 256])

    # Stage 3
    X = convolutional_block(X, filters=[128, 128, 512], s=2)
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])
    X = identity_block(X, filters=[128, 128, 512])

    # Stage 4
    X = convolutional_block(X, filters=[256, 256, 1024], s=2)
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])
    X = identity_block(X, filters=[256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, filters=[512, 512, 2048], s=2)
    X = identity_block(X, filters=[512, 512, 2048])
    X = identity_block(X, filters=[512, 512, 2048])

    # Average Pooling
    X = AveragePooling2D((2, 2))(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = tf.keras.models.Model(inputs=X_input, outputs=X)

    return model



In [11]:
# Create ResNet-50 model with specific input shape
model = ResNet50(input_shape=(48, 48, 3))

# Display model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 48, 48, 3)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 54, 54, 3)            0         ['input_2[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 24, 24, 64)           9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 24, 24, 64)           256       ['conv2d[0][0]']          

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history2 = model.fit(xtrain, ytrain,batch_size=128, epochs=10,
                    validation_data=(xtest, ytest))

Epoch 1/10
469/469 [==============================] - 85s 112ms/step - loss: 0.2453 - accuracy: 0.9313 - val_loss: 0.1418 - val_accuracy: 0.9600
Epoch 2/10
469/469 [==============================] - 51s 109ms/step - loss: 0.0727 - accuracy: 0.9790 - val_loss: 0.2485 - val_accuracy: 0.9247
Epoch 3/10
469/469 [==============================] - 51s 109ms/step - loss: 0.0429 - accuracy: 0.9876 - val_loss: 0.0482 - val_accuracy: 0.9856
Epoch 4/10
469/469 [==============================] - 52s 111ms/step - loss: 0.0386 - accuracy: 0.9883 - val_loss: 0.0421 - val_accuracy: 0.9879
Epoch 5/10
469/469 [==============================] - 51s 110ms/step - loss: 0.0258 - accuracy: 0.9923 - val_loss: 0.0408 - val_accuracy: 0.9874
Epoch 6/10
469/469 [==============================] - 51s 110ms/step - loss: 0.0292 - accuracy: 0.9911 - val_loss: 0.0419 - val_accuracy: 0.9886
Epoch 7/10
469/469 [==============================] - 51s 110ms/step - loss: 0.0237 - accuracy: 0.9927 - val_loss: 0.0408 - val_ac